# Requirements

# Requirements

In [1]:
%pip install transformers
%pip install torch
%pip install pickle5
%pip install mpld3
%pip install scikit-learn
%pip install pattern3
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pickle5 as pickle

def dump_pklfile(file, filepath, size):
	with open(filepath, "wb") as f:
		if (size == 0):
			pickle.dump((file), f)
			return
		if (size > 0):
			pickle.dump((file[:size]), f)
			return
		else:
			pickle.dump((file[size:]), f)
			return
    
def open_pklfile(filepath, size):
	with open(filepath, "rb") as f:
		if (size == 0):
			return pickle.load(f)
		return (pickle.load(f))[0:size]

In [2]:
SINGULAR_NOUN_TEMPLATES = (
    'This is {article} {term}.',
    'That is {article} {term}.',
    'There is {article} {term}.',
    'Here is {article} {term}.',
    'The {term} is here.',
    'The {term} is there.',
)

PLURAL_NOUN_TEMPLATES = (
    'These are {term}.',
    'Those are {term}.',
    'They are {term}.',
    'The {term} are here.',
    'The {term} are there.',
)

TARGET_POSITON=[(3, -2), (3, -2), (3, -2), (3, -2), (2, -4), (2, -4), (3, -2), (3, -2), (3, -2), (2, -4), (2, -4)]


def fill_template(template, term):
    article = (
        'an'
        if (
            (
                term.startswith('honor') or any(
                    term.startswith(c) for c in 'aeiouAEIOU'
                )
            ) and not (
                term.startswith('European') or term.startswith('Ukrainian')
            )
        )
        else 'a'
    )
    sentence = template.format(article=article, term=term)
    return sentence[0].upper() + sentence[1:]



In [3]:
import nltk
from tqdm import tqdm

from pattern.en import pluralize, singularize

def generate_noun_sentences(vocab):
    tags = [(word, tag) for w in vocab for word, tag in nltk.pos_tag([w])]
    nouns = [word for word, tag in tqdm(tags) if tag.startswith("N") and '___' not in word and word is not "_"]
    w2i = {w: i for i, w in enumerate(nouns)}

    sentence_list = []
    for term in tqdm(nouns):
        singular_term = singularize(term)
        sentences = []
        sentences += [fill_template(template, singular_term) for template in SINGULAR_NOUN_TEMPLATES]
        plurar_term = pluralize(term)
        sentences += [fill_template(template, plurar_term) for template in PLURAL_NOUN_TEMPLATES]
        sentence_list.append(sentences)
    return w2i, nouns, sentence_list

# Load pretrained Bert model and tokenizer

In [4]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

debiased_model = "bert"
folder_num = "1."
save_path = f"../data/extracted_positional/{folder_num} {debiased_model}/{debiased_model}_sentence_"

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/home/team4/anaconda3/envs/exp/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactl

# open file and extract bert embedding function

In [5]:
import pickle5 as pickle
from tqdm import tqdm 
import pickle5 as pickle

def dump_pklfile(file, filepath, size):
	with open(filepath, "wb") as f:
		if (size == 0):
			pickle.dump((file), f)
			return
		if (size > 0):
			pickle.dump((file[:size]), f)
			return
		else:
			pickle.dump((file[size:]), f)
			return
    
def open_pklfile(filepath, size):
	with open(filepath, "rb") as f:
		if (size == 0):
			return pickle.load(f)
		return (pickle.load(f))[0:size]

gender_target_list=["male", "males", "man", "men", "boy", "boys", "brother", "brothers", "his", "he", "him", "son", "sons",
              "female", "females", "woman", "women", "girl", "girls", "sister","sisters", "she", "her","hers", "daughter", "daughters"]

def extract_bert_embeddings(sentence_list):
	#init for stacking embeddings
	embeddings = torch.empty(0, device=device)

	for sentences in tqdm(sentence_list):
		embedding = torch.empty(0, device=device)
		for i, sentence in enumerate(sentences):
			# print(f"[{sentence}]")
			# Map the token strings to their vocabulary indeces.
			marked_text = "[CLS] " + sentence + " [SEP]"
			# print(marked_text)
			tokenized_text = tokenizer.tokenize(marked_text)
			# print(tokenized_text)
			# handling such as "wedding_dress"
			tokenized_text = [token for token in tokenized_text if token != '_']

			# Split the sentence into tokens.
			indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
			segments_ids = [1] * len(tokenized_text)

			# Convert inputs to PyTorch tensors
			tokens_tensor = torch.tensor([indexed_tokens], device=device)
			segments_tensors = torch.tensor([segments_ids], device=device)

			# Put the model in "evaluation" mode,meaning feed-forward operation.
			model.eval()

			# Run the text through BERT, get the output and collect all of the hidden states produced from all 12 layers.
			with torch.no_grad():
				outputs = model(tokens_tensor, segments_tensors).hidden_states
				# print(f"outputs: {outputs}")
				start_position=TARGET_POSITON[i][0]
				end_position=TARGET_POSITON[i][1]
				last_four_hidden_states = outputs[-4:]
				concated_hidden_states = torch.cat(last_four_hidden_states, dim=2)
				sentence_embedding=torch.sum(torch.stack([concated_hidden_states[:,start_position], concated_hidden_states[:,end_position-1]]), 0)
				
			embedding = torch.cat([embedding, sentence_embedding], 0)
			if torch.isnan(embedding).any():
				print(sentence)
				print(tokenized_text)
				print(sentence_embedding)
				return

		sum_embedding = torch.sum(embedding, 0)

		norm_embedding = torch.nn.functional.normalize(sum_embedding, dim=0)
		norm_embedding = torch.unsqueeze(norm_embedding, dim=-1)
		norm_embedding = torch.transpose(norm_embedding, 0, 1)


		embeddings = torch.cat([embeddings, norm_embedding], 0)

	return embeddings



In [6]:

def extract_bert_embeddings_gender(sentence_list):
	#init for stacking embeddings
	embeddings = torch.empty(0, device=device)

	for sentences in tqdm(sentence_list):
		embedding = torch.empty(0, device=device)
		for i, sentence in enumerate(sentences):
			# print(f"[{sentence}]")
			# Map the token strings to their vocabulary indeces.
			marked_text = "[CLS] " + sentence + " [SEP]"
			tokenized_text = tokenizer.tokenize(marked_text)
			# handling such as "wedding_dress"
			tokenized_text = [token for token in tokenized_text if token != '_']

			# Split the sentence into tokens.
			indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
			segments_ids = [1] * len(tokenized_text)

			# Convert inputs to PyTorch tensors
			tokens_tensor = torch.tensor([indexed_tokens], device=device)
			segments_tensors = torch.tensor([segments_ids], device=device)

			# Put the model in "evaluation" mode,meaning feed-forward operation.
			model.eval()

			# Run the text through BERT, get the output and collect all of the hidden states produced from all 12 layers.
			with torch.no_grad():
				outputs = model(tokens_tensor, segments_tensors).hidden_states
		
				gender_position=-1
				for idx, t in enumerate(tokenized_text):
					if t in gender_target_list:
						gender_position=idx
						break
				if gender_position==-1:
					print("error")
					print(tokenized_text)
					return 

			
				last_four_hidden_states = outputs[-4:]
				concated_hidden_states = torch.cat(last_four_hidden_states, dim=2)
				sentence_embedding = torch.squeeze(concated_hidden_states)[gender_position]
				sentence_embedding = torch.unsqueeze(sentence_embedding, dim=0)
    
				
			embedding = torch.cat([embedding, sentence_embedding], 0)
	
			if torch.isnan(embedding).any():
				print(sentence)
				print(tokenized_text)
				print(sentence_embedding)
				return


		sum_embedding = torch.sum(embedding, 0)
		norm_embedding = torch.nn.functional.normalize(sum_embedding, dim=0)
		norm_embedding = torch.unsqueeze(norm_embedding, dim=0)
		# norm_embedding = torch.transpose(norm_embedding, 0, 1)

		embeddings = torch.cat([embeddings, norm_embedding], 1)

	return embeddings



# computing bert bias function

In [7]:

import numpy as np
from torch import linalg as LA
import scipy.stats
import json 
import codecs

# normalize vectors
def normalize(wv):    
    norms = LA.norm(wv, dim=1)
    wv = wv / norms[:, np.newaxis]
    return wv

# compute bias from bert with he-she
def compute_bias_by_projection_sentence(vocab, lim_wv, gender_word_embedding):
    # print(gender_word_embedding[0].shape)
    print(lim_wv)
    males = torch.tensordot(lim_wv, gender_word_embedding[0], dims=1)
    females = torch.tensordot(lim_wv, gender_word_embedding[1], dims=1)
    
    d = {}
    for w, m, f in tqdm(zip(vocab, males, females)):
        d[w] = m - f
    return d

def extract_professions():
    professions = []
    with codecs.open('../data/lists/professions.json', 'r', 'utf-8') as f:
        professions_data = json.load(f)
    for item in professions_data:
        professions.append(item[0].strip())
    return professions

## Extracting Embeddings: 2016 words + bert + sentence

In [26]:
vocab = open_pklfile("../data/extracted/0. original/original_word_2016_restricted_vocab.pkl", 0)
w2i, nouns, sentence_list = generate_noun_sentences(vocab)
lim_wv = extract_bert_embeddings(sentence_list)
dump_pklfile(nouns, f"{save_path}2016_restricted_vocab.pkl", 0)
dump_pklfile(lim_wv, f"{save_path}2016_restricted_embeddings.pkl", 0)
lim_wv = normalize(lim_wv)


100%|██████████| 18445/18445 [22:05<00:00, 13.91it/s]


In [28]:
with open("../data/lists/male_sentence_file.txt", 'r') as f:
    male_sentences = [sentence.strip() for sentence in f.readlines()]

with open("../data/lists/female_sentence_file.txt", 'r') as f:
    female_sentences = [sentence.strip() for sentence in f.readlines()]

male_embeddings = extract_bert_embeddings_gender([male_sentences])
female_embeddings = extract_bert_embeddings_gender([female_sentences])
# # print(male_embeddings.shape)
male_embeddings = normalize(male_embeddings)
female_embeddings = normalize(female_embeddings)

lim_wv= normalize(open_pklfile("../data/extracted_positional/1. bert/bert_sentence_2016_restricted_embeddings.pkl", 0))
print(lim_wv.shape)

gender_word_embedding = torch.cat((male_embeddings, female_embeddings), 0)



gender_bias_all = compute_bias_by_projection_sentence(nouns, lim_wv, gender_word_embedding)


100%|██████████| 1/1 [00:00<00:00,  1.94it/s]


torch.Size([18445, 3072])
tensor([[ 0.0199, -0.0264,  0.0297,  ..., -0.0131,  0.0085, -0.0414],
        [ 0.0035, -0.0201,  0.0216,  ..., -0.0122,  0.0085,  0.0075],
        [ 0.0238,  0.0003,  0.0193,  ..., -0.0204,  0.0008, -0.0056],
        ...,
        [ 0.0528, -0.0175,  0.0194,  ..., -0.0136, -0.0009,  0.0010],
        [ 0.0377, -0.0423,  0.0124,  ..., -0.0163,  0.0119,  0.0038],
        [ 0.0262, -0.0439,  0.0295,  ..., -0.0116,  0.0130, -0.0101]],
       device='cuda:0')


18445it [00:00, 162419.07it/s]


In [29]:
import operator

sorted_g = sorted(gender_bias_all.items(), key=operator.itemgetter(1))
females_w = [item[0] for item in sorted_g[:2500]]
females_e = torch.empty(0, device=device)
for w in females_w:
    temp = torch.unsqueeze(lim_wv[w2i[w]], 0)
    females_e = torch.cat([females_e, temp], dim=0)

dump_pklfile(females_w, f"{save_path}2016_female_2500_vocab.pkl", 2500)
dump_pklfile(females_e, f"{save_path}2016_female_2500_embeddings.pkl", 2500)

sorted_g = sorted(gender_bias_all.items(), key=operator.itemgetter(1), reverse=True)
males_w = [item[0] for item in sorted_g[:2500]]
males_e = torch.empty(0, device=device)
for w in males_w:
    temp = torch.unsqueeze(lim_wv[w2i[w]], 0)
    males_e = torch.cat([males_e, temp], dim=0)

dump_pklfile(males_w, f"{save_path}2016_male_2500_vocab.pkl", 2500)
dump_pklfile(males_e, f"{save_path}2016_male_2500_embeddings.pkl", 2500)

## Extracting Embeddings: 2018 words + bert + sentence

In [8]:
vocab = open_pklfile("../data/extracted/0. original/original_word_2018_restricted_vocab.pkl", 0)
w2i, nouns, sentence_list = generate_noun_sentences(vocab)
lim_wv = extract_bert_embeddings(sentence_list)
dump_pklfile(nouns, f"{save_path}2018_restricted_vocab.pkl", 0)
dump_pklfile(lim_wv, f"{save_path}2018_restricted_embeddings.pkl", 0)

lim_wv = normalize(lim_wv)

100%|██████████| 39385/39385 [49:49<00:00, 13.18it/s]


In [9]:
with open("../data/lists/male_sentence_file.txt", 'r') as f:
  male_sentences = [sentence.strip() for sentence in f.readlines()]

with open("../data/lists/female_sentence_file.txt", 'r') as f:
  female_sentences = [sentence.strip() for sentence in f.readlines()]

male_embeddings = extract_bert_embeddings_gender([male_sentences])
female_embeddings = extract_bert_embeddings_gender([female_sentences])

male_embeddings = normalize(male_embeddings)
female_embeddings = normalize(female_embeddings)

gender_word_embedding = torch.cat((male_embeddings, female_embeddings), 0)

gender_bias_all = compute_bias_by_projection_sentence(nouns, lim_wv, gender_word_embedding)

100%|██████████| 1/1 [00:00<00:00,  1.86it/s]


tensor([[ 1.0368e-02, -1.0730e-02,  1.5928e-03,  ..., -7.4427e-03,
          1.5947e-03,  1.5316e-02],
        [ 3.7644e-02,  4.4513e-03, -7.6410e-04,  ..., -1.6967e-02,
          1.1548e-02,  1.8938e-04],
        [-1.7022e-04, -3.0128e-02,  2.7313e-02,  ..., -1.1867e-02,
          1.6743e-05, -1.0438e-02],
        ...,
        [ 1.7373e-02, -1.0948e-02, -2.6629e-03,  ..., -2.1898e-02,
          4.6303e-03, -1.9816e-02],
        [ 9.9961e-03, -1.7308e-02,  3.7393e-04,  ..., -2.2310e-02,
          4.2690e-03,  1.5567e-02],
        [ 1.3339e-02, -1.1929e-03,  7.9271e-03,  ..., -1.2481e-02,
          1.5983e-02,  2.8010e-02]], device='cuda:0')


39385it [00:00, 163960.66it/s]


In [10]:
import operator

sorted_g = sorted(gender_bias_all.items(), key=operator.itemgetter(1))
females_w = [item[0] for item in sorted_g[:2500]]
females_e = torch.empty(0, device=device)

for w in females_w:
    temp = torch.unsqueeze(lim_wv[w2i[w]], 0)
    females_e = torch.cat([females_e, temp], dim=0)

dump_pklfile(females_w, f"{save_path}2018_female_2500_vocab.pkl", 2500)
dump_pklfile(females_e, f"{save_path}2018_female_2500_embeddings.pkl", 2500)

sorted_g = sorted(gender_bias_all.items(), key=operator.itemgetter(1), reverse=True)
males_w = [item[0] for item in sorted_g[:2500]]
males_e = torch.empty(0, device=device)

for w in males_w:
    temp = torch.unsqueeze(lim_wv[w2i[w]], 0)
    males_e = torch.cat([males_e, temp], dim=0)

dump_pklfile(males_w, f"{save_path}2018_male_2500_vocab.pkl", 2500)
dump_pklfile(males_e, f"{save_path}2018_male_2500_embeddings.pkl", 2500)

### extract word file embeddings

In [31]:
with open("../data/lists/male_sentence_file.txt", 'r') as f:
  male_sentences = [sentence.strip() for sentence in f.readlines()]
male_sentence_embs = extract_bert_embeddings_gender([male_sentences])
dump_pklfile(male_sentence_embs, f"{save_path}male_word_file_embeddings.pkl", 0)

with open("../data/lists/female_sentence_file.txt", 'r') as f:
  female_sentences = [sentence.strip() for sentence in f.readlines()]
female_sentence_embs = extract_bert_embeddings_gender([female_sentences])
dump_pklfile(female_sentence_embs, f"{save_path}female_word_file_embeddings.pkl", 0)

100%|██████████| 1/1 [00:00<00:00,  1.94it/s]
